<a href="https://colab.research.google.com/github/MohammadNPak/MFTDjango04/blob/master/hw3Q1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [92]:
!pip install torchinfo
import torch
import torchvision
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from torchinfo import summary
from sklearn.model_selection import train_test_split

device = "cuda" if torch.cuda.is_available() else "cpu"


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [93]:
# !unzip /content/drive/MyDrive/NNDL/HW3/Q1/archive_flowers.zip -d ./
from pathlib import Path
from sklearn.preprocessing import LabelBinarizer
from torch.utils.data import Dataset,DataLoader
from torchvision.io import read_image

weights = torchvision.models.AlexNet_Weights.DEFAULT
transform = weights.transforms()

class FlowerDataset(Dataset):
    def __init__(self,img_labels,img_dirs,transform=None,target_transform=None):
        self.img_dirs = img_dirs
        self.img_labels = img_labels
        self.transform = transform
        self.target_transform = target_transform
    
    def __len__(self):
        return len(self.img_dirs)
    
    def __getitem__(self, index):
        image = read_image(str(self.img_dirs.iloc[index,0]))
        label = torch.tensor(self.img_labels[index],dtype=float)

        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image,label


dataset_dir = Path('flowers')
data = pd.DataFrame(dataset_dir.glob('*/*.*'),columns=['image_path'])
label_str = data['image_path'].apply(lambda x:str(x.parent.name))
enc = LabelBinarizer()
enc.fit(label_str)
label = enc.transform(label_str)

X_train,X_test,Y_train,Y_test =train_test_split(
    data,
    label,
    train_size=0.8,
    random_state=0,
    shuffle=True)

train_dataset = FlowerDataset(Y_train,X_train,transform=transform)
test_dataset = FlowerDataset(Y_test,X_test,transform=transform)
train_loader =DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader =DataLoader(test_dataset, batch_size=16, shuffle=True)


In [42]:
str(X_train.iloc[0,0])
train_dataset[0][1].dtype

torch.float64

In [94]:

model = torchvision.models.alexnet(weights=weights).to(device)
print(model)
print(summary(model=model,
        input_size=(32, 3, 224, 224), # make sure this is "input_size", not "input_shape"
        col_names=["input_size","trainable"], # uncomment for smaller output
        # col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]))

for param in model.features.parameters():
    param.requires_grad = False

print(summary(model=model,
        input_size=(32, 3, 224, 224), # make sure this is "input_size", not "input_shape"
        col_names=["input_size","trainable"], # uncomment for smaller output
        col_width=15,
        row_settings=["var_names"]))

model.classifier = torch.nn.Sequential(
    torch.nn.Dropout(p=0.2, inplace=True), 
    torch.nn.Linear(in_features=9216,out_features=128,bias=True),
    torch.nn.Dropout(p=0.2, inplace=True), 
    torch.nn.ReLU(inplace=True),
    torch.nn.Linear(in_features=128,out_features=5,bias=True),
    torch.nn.Dropout(p=0.2, inplace=True), 
    torch.nn.ReLU(inplace=True),
    torch.nn.Softmax()
    ).to(device)

model = model.to(device)
print(summary(model=model,
        input_size=(32, 3, 224, 224), # make sure this is "input_size", not "input_shape"
        col_names=["input_size","trainable"], # uncomment for smaller output
        col_width=15,
        row_settings=["var_names"]))


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

/usr/local/lib/python3.8/dist-packages/torch/nn/modules/container.py:139: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


In [97]:
from torch.nn import CrossEntropyLoss
from torch.optim import Adam,SGD

optimizer = Adam(model.parameters())
loss_fn = CrossEntropyLoss()

def train_one_epoch(epoch_index, tb_writer):
    running_loss = 0.
    last_loss = 0.
    for i, data in enumerate(train_loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:
            last_loss = running_loss / 100 # loss per batch
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            tb_x = epoch_index * len(train_loader) + i + 1
            tb_writer.add_scalar('Loss/train', last_loss, tb_x)
            running_loss = 0.
    return last_loss



In [99]:
from datetime import datetime
from torch.utils.tensorboard import SummaryWriter
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('trainer_{}'.format(timestamp))

epoch_number = 0
EPOCHS = 50
best_vloss = 1_000_000.
for epoch in range(EPOCHS):
    print('EPOCH {}:'.format(epoch_number + 1))
    model.train(True)
    avg_loss = train_one_epoch(epoch_number, writer)
    model.train(False)
    running_vloss = 0.0
    for i, vdata in enumerate(test_loader):
        vinputs, vlabels = vdata
        vinputs, vlabels = vinputs.to(device), vlabels.to(device)
        voutputs = model(vinputs)
        vloss = loss_fn(voutputs, vlabels)
        running_vloss += vloss

    avg_vloss = running_vloss / (i + 1)
    print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))

    writer.add_scalars('Training vs. Validation Loss',
                    { 'Training' : avg_loss, 'Validation' : avg_vloss },
                    epoch_number + 1)
    writer.flush()

    if avg_vloss < best_vloss:
        best_vloss = avg_vloss
        model_path = 'model_{}_{}'.format(timestamp, epoch_number)
        torch.save(model.state_dict(), model_path)

    epoch_number += 1


EPOCH 1:
  batch 100 loss: 1.3136874494329094
  batch 200 loss: 1.3060771976038814
LOSS train 1.3060771976038814 valid 1.23096266588955
EPOCH 2:
  batch 100 loss: 1.2777340744808316
  batch 200 loss: 1.2984785130620002
LOSS train 1.2984785130620002 valid 1.190324119257706
EPOCH 3:
  batch 100 loss: 1.271614140495658
  batch 200 loss: 1.279600830897689
LOSS train 1.279600830897689 valid 1.193011433231058
EPOCH 4:
  batch 100 loss: 1.2411897188425065
  batch 200 loss: 1.248163973838091
LOSS train 1.248163973838091 valid 1.1752992962935456
EPOCH 5:
  batch 100 loss: 1.2436519037932157
  batch 200 loss: 1.2479689831286669
LOSS train 1.2479689831286669 valid 1.1844841935154464
EPOCH 6:
  batch 100 loss: 1.2385349667444825
  batch 200 loss: 1.2498555636033415
LOSS train 1.2498555636033415 valid 1.1554983974330955
EPOCH 7:
  batch 100 loss: 1.230844008885324
  batch 200 loss: 1.2357638010010124
LOSS train 1.2357638010010124 valid 1.1813380378125993
EPOCH 8:
  batch 100 loss: 1.228962343297898

KeyboardInterrupt: ignored

In [98]:
model.eval()
# d = train_dataset[1][0].to(device)
i = iter(train_loader)
d = next(i)
print(model(d[0].to(device)))
print(d[1])
# print(type(train_dataset[0:2][0][:,:,:]))
# print(d.shape)

tensor([[2.8918e-04, 2.8918e-04, 2.8918e-04, 9.9884e-01, 2.8918e-04],
        [2.2158e-06, 2.2158e-06, 5.7251e-06, 2.2158e-06, 9.9999e-01],
        [7.1381e-02, 7.1448e-01, 7.1381e-02, 7.1381e-02, 7.1381e-02],
        [5.0541e-04, 9.9798e-01, 5.0541e-04, 5.0541e-04, 5.0541e-04],
        [1.9520e-07, 1.0000e+00, 1.9520e-07, 1.9520e-07, 1.9520e-07],
        [7.7221e-13, 7.7221e-13, 9.9981e-01, 7.7221e-13, 1.8798e-04],
        [6.7882e-09, 6.7882e-09, 1.0000e+00, 6.7882e-09, 6.7882e-09],
        [9.2142e-07, 9.2142e-07, 1.0000e+00, 9.2142e-07, 9.2142e-07],
        [2.9117e-12, 2.9117e-12, 2.9117e-12, 1.0000e+00, 2.9117e-12],
        [5.4336e-07, 1.0000e+00, 5.4336e-07, 5.4336e-07, 5.4336e-07],
        [2.2226e-02, 9.1109e-01, 2.2226e-02, 2.2226e-02, 2.2226e-02],
        [2.5465e-05, 9.9990e-01, 2.5465e-05, 2.5465e-05, 2.5465e-05],
        [6.1373e-08, 6.3270e-07, 1.0000e+00, 6.1373e-08, 6.1373e-08],
        [1.6577e-03, 1.6577e-03, 1.6577e-03, 9.9337e-01, 1.6577e-03],
        [9.4885e-06,